# Imports

In [3]:
import numpy as np
import jax
from jax import jacfwd, jacrev
import jax.numpy as jnp
from jax import grad
from pprint import pprint as pp
import re #Used to read chunks from the data files - clusters, vmat, etec.
import math
from tqdm.notebook import tqdm
import pandas as pd
from datetime import datetime
import random
from IPython.display import clear_output
import itertools
import sys
from scipy.linalg import eigvals

import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from mpl_toolkits.mplot3d import Axes3D
import plotly.figure_factory as ff
import matplotlib.tri as tri

from scipy.spatial import Delaunay
from scipy.optimize import minimize, basinhopping
from scipy.optimize import SR1, BFGS
from scipy.optimize import Bounds
from scipy.optimize import LinearConstraint
from scipy.optimize import NonlinearConstraint

from ase import Atoms
from ase.units import kB
from ase.visualize import view
from ase.build import bulk
from ase.spacegroup import crystal
from ase.build import make_supercell
from ase.io.vasp import write_vasp

import sys
import subprocess
import os

pattern1 = re.compile("\n\n\n")
pattern2 = re.compile("\n\n")
np.set_printoptions(suppress=True,precision=3)
random.seed(42)
np.random.seed(seed=42)

# Read all necessary files

## Read clusters.out

In [4]:
# Read clusters.out
clusters = {}

with open('BCC_A2/sqs_lev=3_a_Mo=0.33333,a_V=0.33333,a_W=0.33333/clusters.out','r') as fclusters:
    temp_clusters = fclusters.read().split('\n\n') #Read blocks separated by 1 empty line

for idx, cluster in enumerate(temp_clusters):
    if cluster == '': #Check for spurious empty blocks
        continue 
    line = cluster.split('\n') #If not empty split by lines
    multiplicity = int(line[0]) #1st line
    length = float(line[1]) #largest distance between two atoms
    num_points = int(line[2]) #type of cluster
    clusters[idx] = {'mult':multiplicity, 'length':length, 'type':num_points}

## Read config.out

In [5]:
# Read config.out
configs = {}

fconfig = open('BCC_A2/sqs_lev=3_a_Mo=0.33333,a_V=0.33333,a_W=0.33333/config.out','r')
_ = next(fconfig) #Ignore first line

temp_config = fconfig.read()#.split('\n\n')

temp_config = pattern1.split(temp_config) #split lines separated by 2 empty lines

for idx, config in enumerate(temp_config):
    if config == '': #Check for spurious empty blocks
        continue
    num_points = int(config.split('\n')[0]) #number of subclusters
    inter = []
    config = pattern2.split(config) #now split individual subclusters separated by 1 blank line
    for i in range(num_points):
        line = config[i].split('\n')
        if i == 0:
            length = int(line[1])
        else:
            length = int(line[0])
        tmp_inter = np.array([(list(map(int,l.split(' ')[-2:]))) for l in line[-length:]])
        inter.append(tmp_inter)#print(np.array(inter))
    configs[idx] = {'inter': inter, 'num_of_subclus': num_points}

## Read Kikuchi-Baker coefficients

In [6]:
# Read kb.out

kb = {}
fkb = open('BCC_A2/sqs_lev=3_a_Mo=0.33333,a_V=0.33333,a_W=0.33333/configkb.out','r')
_ = next(fkb) #ignore first line

temp_kb = fkb.read()
temp_kb = temp_kb.split('\n') #split file linewise

for idx, kbcoeff in enumerate(temp_kb):
    if kbcoeff == '': #check for spurious empty blocks
        continue
    kb[idx] = float(kbcoeff)

fkb.close()

## Read coefficients of subclusters

In [7]:
# Read configcoeff.out
configcoef = {}

with open('BCC_A2/sqs_lev=3_a_Mo=0.33333,a_V=0.33333,a_W=0.33333/configmult.out','r') as fsubmult:
    _ = next(fsubmult) #ignore first line
    temp_submult = fsubmult.read() 
    temp_submult = pattern2.split(temp_submult) #split lines into blocks separated by 2 empty lines
    
for idx, submult in enumerate(temp_submult):
    submult = submult.split('\n') #split into number of subclusters 
    while("" in submult) :
        submult.remove("") #remove empty blocks
    configcoef[idx] = list(map(float,submult[1:])) #also ignore 1st line of each block
configcoef.pop(list(configcoef.keys())[-1])

[]

## Read V-Matrix

In [8]:
# Read vmat.out
vmat = {}
with open('BCC_A2/sqs_lev=3_a_Mo=0.33333,a_V=0.33333,a_W=0.33333/vmat.out') as fvmat:
    _ = next(fvmat) #ignore first lie
    temp_vmat = fvmat.read()
    temp_vmat = pattern2.split(temp_vmat) #split by 2 empty lines i.e. maxclusters
    
    while("" in temp_vmat):
        temp_vmat.remove("") #remove empty blocks
    
    for clus_idx, mat in enumerate(temp_vmat):
        mat = mat.split('\n') #split by 1 empty line i.e. subclusters
        mat_float = np.empty(list(map(int, mat[0].split(' '))))
        for idx, row in enumerate(mat[1:]): #ignore first line
            mat_float[idx] = list(map(float,row.split(' ')[:-1]))
        
        vmat[clus_idx] = mat_float

## Read Cluster Mult

In [9]:
clustermult = {}

with open('BCC_A2/sqs_lev=3_a_Mo=0.33333,a_V=0.33333,a_W=0.33333/clusmult.out', 'r') as fcm:
    _ = next(fcm)  # Ignore first line
    temp_mult = fcm.read()
    temp_mult = temp_mult.split('\n')  # split by line

for idx, mult in enumerate(temp_mult):
    if mult == '':
        continue
    clustermult[idx] = float(mult)

## Read ECI

In [10]:
# Read eci
eci = {}

with open('BCC_A2/sqs_lev=3_a_Mo=0.33333,a_V=0.33333,a_W=0.33333/eci_tetrahedron.out') as feci:
    _ = next(feci) #Ignore first line
    temp_eci = feci.read()
    temp_eci = temp_eci.split('\n') #split by line

for idx, eci_val in enumerate(temp_eci):
    if eci_val == '':
        continue
    eci[idx] = float(eci_val)

In [11]:
#In this notebook ECI's are declared manually
eci_2 = 0.01
eci_3 = 2*eci_2
eci = {0: 0, 1: 0.0, 2: 0.04, 3: 0, 4: 0, 5: 0}

In [11]:
corrs1 = np.array([1., 1., 1., 1., 1., 1.]) #Pure B
corrs0 = np.array([1., -1., 1., 1., -1., 1.]) #Pure A
corrssqs = np.array([1.    , 0.0   , 0.25,0.25  , 0.125 , 0.0625]) # AB - sqs
corrsrand = np.array([1.   , *np.random.uniform(-1, 1, 5)]) 
corrs_rnd = np.array([1.    , 0.0   , 0.0 , *np.random.uniform(-1, 1, 18)]) # AB - sqs
T = 100

In [11]:
corrs_disordered = np.array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,  0., 0., 0., 0., 0., 0., 0.,  0.])                              
corrs_ordered = np.array([1., 0., 0., -0.3125, 0.6135, -0.5207, -0.3747, -0.577,  0.1245, 0.0003,  
                          0.0358, -0.3749, 0.1439, 0.604, -0.3614,  
                          1., -0.4336,  1., -0.333,  -0.1433, -1.]) 
corrs_100_nocons = np.array([1., 0.,     -0.,     -0.1701,  0.5731, -0.3322, -0.027,  -0.2143, -0.0945,
                             0.155,  -0.1076, -0.2022,  0.1369,  0.3075, -0.4804,  0.3786, -0.205,
                             0.3327, 0.0983, -0.169,  -0.9824])
corrs_1000_nocons = np.array([1.,      0.,      0.,     -0.1366,  0.4484, -0.2673, -0.0892,  0.0189, -0.1465,
                              0.1764, -0.0639, -0.0891,  0.0927,  0.2353, -0.5212,  0.4689, -0.2016,  0.2252,
                              0.049,  -0.088,  -0.9813])
corrs_100_cons = np.array([ 1.,      0.,     -0.,    -0.1362,  0.5376, -0.3492, -0.0616, -0.2397, -0.2259,
                           0.1227, -0.1948, -0.2146,  0.1637,  0.3222, -0.4231,  0.1585, -0.1197,  0.1382,
                           0.1078, -0.0774, -0.5255])
corrs_1000_cons = np.array([1., 0., 0., -0.1383,  0.4167, -0.3007, -0.0989,  0.0098, -0.1904,
                       0.1229, -0.1023, -0.0726,  0.0829,  0.2475, -0.474,  0.2463, -0.1156,  0.1298, 
                       0.0672, -0.048,  -0.7431])
corrs_ordered_limited = np.array([ 1.,     0.,    -0.,     -0.4285,  0.2758, -0.4911,  0.5712, -0.4949,  0.0178,
                                  -0.1429, -0.7139, -0.6248,  1.,      0.2455,  0.2781, -1.,      1.,     -1.,
                                  0.9821, -0.6803,  1.    ])


In [13]:
corrs_test = np.array([ 1.,     -0.,     -0.,     -0.2919,  0.5849, -0.3281,  0.0351, -0.2588, -0.0758,
                       0.143,  -0.0185, -0.2273,  0.0202,  0.3161, -0.5061,  0.6908, -0.4132,  0.5505,
                       0.051,  -0.0606, -0.9868])


In [23]:
corrs_rnd = subprocess.run(['/home/sayan/bin/corrdump', '-c', 
                       '-cf=BCC_A2/sqs_lev=3_a_Mo=0.33333,a_V=0.33333,a_W=0.33333/clusters.out', 
                       '-s=BCC_A2/sqs_lev=3_a_Mo=0.33333,a_V=0.33333,a_W=0.33333/ecifit_9/str.in', 
                       '-l=BCC_A2/sqs_lev=3_a_Mo=0.33333,a_V=0.33333,a_W=0.33333/lat.in'],
                      stdout=subprocess.PIPE,
                      stderr=subprocess.PIPE,
                      check=True
                     )
corrs_rnd = corrs_rnd.stdout.decode('utf-8').split('\t')[:-1]
corrs_rnd = np.array(corrs_rnd, dtype=np.float32)  # convert to arrays
corrs_rnd

array([ 1.     ,  0.     , -0.     , -0.03906,  0.04059, -0.00781,
        0.0625 ,  0.     ,  0.     ,  0.00586, -0.00338, -0.0293 ,
       -0.02368, -0.00586, -0.00338,  0.02734, -0.0203 , -0.01172,
        0.00781,  0.     , -0.00391], dtype=float32)

In [24]:
def check_result_validity(corrs):
    try:
        for config_idx in configcoef:
            assert np.isclose(np.inner(configcoef[config_idx], vmat[config_idx] @ corrs),
                              1.0,
                              rtol=1e-3,
                              atol=1e-9
                              )
    except AssertionError:
        return False

    return True

while True:
    #corrs0 = np.array([1.    , 0.0   , 0.0 , *np.random.normal(0, 0.01, 18)])
    corrs_rnd = np.array([1.    , 0.0   , 0.0 , *np.random.normal(0, 0.1, 18)])
    if check_result_validity(corrs_rnd):
        print(corrs_rnd)
        break

[ 1.        0.        0.        0.049671 -0.013826  0.064769  0.152303
 -0.023415 -0.023414  0.157921  0.076743 -0.046947  0.054256 -0.046342
 -0.046573  0.024196 -0.191328 -0.172492 -0.056229 -0.101283  0.031425]


In [27]:
corrs_rnd = np.array([1.    , 0.0   , 0.0 , *np.random.normal(0, 0.1, 18)])

In [14]:
print('Disordered')
for vm in vmat.values():
    #print(f"{' '.join(map(str, vm @ corrs_ordered))}")
    print(vm @ corrs_disordered)
print()
print('Ordered')
for vm in vmat.values():
    #print(f"{' '.join(map(str, vm @ corrs_ordered))}")
    print(vm @ corrs_ordered)
print()
print('limitd')
for vm in vmat.values():
    #print(f"{' '.join(map(str, vm @ corrs_ordered))}")
    print(vm @ corrs_ordered_limited)
# print('100 constrained')
# for vm in vmat.values():
#     #print(f"{' '.join(map(str, vm @ corrs_ordered))}")
#     print(vm @ corrs_100_cons)
# print()
# print('100 not constrained')
# for vm in vmat.values():
#     #print(f"{' '.join(map(str, vm @ corrs_ordered))}")
#     print(vm @ corrs_100_nocons)
# print()
# print('1000 not constrained')
# for vm in vmat.values():
#     #print(f"{' '.join(map(str, vm @ corrs_ordered))}")
#     print(vm @ corrs_1000_nocons)
# print()
# print('1000 constrained')
# for vm in vmat.values():
#     #print(f"{' '.join(map(str, vm @ corrs_ordered))}")
#     print(vm @ corrs_1000_cons)
# print()
print('test')
for vm in vmat.values():
    #print(f"{' '.join(map(str, vm @ corrs_ordered))}")
    print(vm @ corrs_test)


Disordered
[1.]
[0.333 0.333 0.333]
[0.111 0.111 0.111 0.111 0.111 0.111]
[0.111 0.111 0.111 0.111 0.111 0.111]
[0.037 0.037 0.037 0.037 0.037 0.037 0.037 0.037 0.037 0.037 0.037 0.037
 0.037 0.037 0.037 0.037 0.037 0.037]
[0.012 0.012 0.012 0.012 0.012 0.012 0.012 0.012 0.012 0.012 0.012 0.012
 0.012 0.012 0.012 0.012 0.012 0.012 0.012 0.012 0.012]

Ordered
[1.]
[0.333 0.333 0.333]
[0.076 0.188 0.069 0.    0.146 0.118]
[0.069 0.076 0.187 0.167 0.09  0.056]
[ 0.     0.038  0.038  0.118  0.031  0.     0.069 -0.     0.118  0.
  0.     0.028  0.     0.038  0.031  0.049  0.059  0.028]
[ 0.     0.     0.     0.069  0.     0.     0.     0.038 -0.     0.
  0.    -0.     0.049  0.031 -0.     0.     0.     0.     0.     0.028
 -0.   ]

limitd
[1.]
[0.333 0.333 0.333]
[0.063 0.161 0.108 0.032 0.14  0.085]
[0.175 0.032 0.127 0.176 0.125 0.081]
[0.032 0.032 0.    0.088 0.042 0.067 0.057 0.    0.104 0.    0.032 0.004
 0.086 0.    0.023 0.088 0.052 0.01 ]
[-0.     0.032  0.     0.     0.025  0.06  -

In [141]:
configcoef

{0: [1.0],
 1: [1.0, 1.0, 1.0],
 2: [1.0, 2.0, 2.0, 1.0, 2.0, 1.0],
 3: [1.0, 2.0, 2.0, 1.0, 2.0, 1.0],
 4: [1.0,
  2.0,
  2.0,
  1.0,
  2.0,
  1.0,
  1.0,
  2.0,
  2.0,
  1.0,
  2.0,
  1.0,
  1.0,
  2.0,
  2.0,
  1.0,
  2.0,
  1.0],
 5: [1.0,
  4.0,
  4.0,
  2.0,
  4.0,
  2.0,
  4.0,
  8.0,
  4.0,
  8.0,
  4.0,
  4.0,
  4.0,
  8.0,
  4.0,
  1.0,
  4.0,
  2.0,
  4.0,
  4.0,
  1.0]}

# Setting up Log Absolute functions

## Set up F

### Numpy

In [63]:
mult_arr = np.array(list(clustermult.values()))
eci_arr = np.array(list(eci.values()))

all_vmat = np.vstack([vmat for vmat in vmat.values()])
mults_config = np.asarray(list(itertools.chain.from_iterable(list(configcoef.values()))))
mults_eci = np.multiply(mult_arr,eci_arr)
all_kb = np.array(list(itertools.chain.from_iterable([[kb for _ in range(len(configcoef[idx]))] for idx, kb in kb.items()])))

multconfig_kb = np.multiply(mults_config,all_kb)

rhologrho = lambda rho: rho * np.log(np.abs(rho))
vrhologrho = np.vectorize(rhologrho)

In [64]:
def F(corrs, vmat, kb, clusters, configs, configcoef,T,eci):
    """
    Input: 
    corrs - Correlations
    vmat  - V-Matrix
    clusters - Maximal Cluster Information (multiplicity, longest neighbor length, no. of points)
    configs - Not used
    configcoef - Coefficients of subclusters - array containing the coeff of each subcluster
    T - Temperature
    eci - ECI's
    mu - chemical potentials
    
    Output:
    F = H + kB*T*SUM(rho * log(rho))
    """
    
    def get_corrsum(vmat,corrs):
        assert len(vmat) == len(corrs)
        corrsum = np.inner(vmat,corrs)
        if corrsum == 0:
            return 0
            #corrsum = np.finfo(float).tiny

        return corrsum * np.log(np.abs(corrsum))
    
    def per_cluster_sum(corrs,vmat,configcoef):
        config_sum = np.sum([coef * get_corrsum(vmat[config_idx],corrs) for config_idx, coef in enumerate(configcoef)
                            ])
                      
        return config_sum
    
    H = np.sum([cluster['mult']*eci[cluster_idx]*corrs[cluster_idx] 
                for cluster_idx, cluster in clusters.items()
               ])
    
    S = np.sum([kb[cluster_idx]*per_cluster_sum(corrs,
                                                vmat[cluster_idx],
                                                configcoef[cluster_idx],)
                for cluster_idx in configs.keys()
               ])
    
    #rho1 = np.matmul(vmat[1],corrs) #mole fractions
    
    return H + kB*T*S #- np.sum(mu*rho1)

def F_optim_np(corrs, vmat, kb, clusters,clustermult, configs, configcoef,T,eci):

    H = mults_eci @ corrs
    #S = np.inner(multconfig_kb,vrhologrho(all_vmat @ corrs))
    S = multconfig_kb @ vrhologrho(all_vmat @ corrs)
    
    return H + kB*T*S #- np.sum(mu*rho1)
    #return jnp.dot(mults_eci,corrs) + kB*T*jnp.vdot(multconfig_kb,vrhologrho(jnp.dot(all_vmat, corrs)))

In [65]:
#f0 = F(corrs0, vmat, kb, clusters, configs, configcoef,T,eci)
#f1 = F(corrs1, vmat, kb, clusters, configs, configcoef,T,eci)
#fsqs = F(corrssqs, vmat, kb, clusters, configs, configcoef,T,eci)
fnew = F_optim_np(corrs_rnd, vmat, kb, clusters, clustermult, configs, configcoef,100,eci)
#frand = F(corrsrand, vmat, kb, clusters, configs, configcoef,T,eci)
frnd = F(corrs_rnd, vmat, kb, clusters, configs, configcoef,100,eci)
#print(f"Corrs 0: {f0:.2f} -- Corrs 1: {f1:.2f} -- Corrs Rand: {frand:.2f} -- Corrs SQS: {fsqs:.2f} -- Corrs New: {fnew:.2f} -- Corrs RND: {frnd:.2f}")

In [138]:
T = 2000
f_disordered_100 = F(corrs_disordered, vmat, kb, clusters, configs, configcoef,100,eci)
f_ordered_100 = F(corrs_ordered, vmat, kb, clusters, configs, configcoef,0,eci)
f_disordered_1000 = F(corrs_disordered, vmat, kb, clusters, configs, configcoef,1000,eci)
f_ordered_1000 = F(corrs_ordered, vmat, kb, clusters, configs, configcoef,1000,eci)
f_cons_100 = F(corrs_100_cons, vmat, kb, clusters, configs, configcoef,100,eci)
f_nocons_100 = F(corrs_100_nocons, vmat, kb, clusters, configs, configcoef,100,eci)
f_cons_1000 = F(corrs_1000_cons, vmat, kb, clusters, configs, configcoef,1000,eci)
f_nocons_1000 = F(corrs_1000_nocons, vmat, kb, clusters, configs, configcoef,1000,eci)

In [120]:
o2d = np.linalg.norm((corrs_disordered - corrs_ordered),ord=2)
o2d

2.282847802635997

In [121]:
o2d/2 - np.linalg.norm(-1*(corrs_disordered - corrs_1000_nocons)), o2d/2 - np.linalg.norm(-1*(corrs_disordered - corrs_1000_cons))

(-0.26712670480163947, -2.1103008009948354e-06)

In [122]:
f_cons_1000, f_nocons_1000

(-0.5597072146604241, -0.5626475872046909)

In [140]:
f_disordered_1000, f_ordered_100

(-0.455926768380781, -0.23778639890000003)

## Set up F Jacobian

In [29]:
def F_jacobian(corrs, vmat, kb, clusters, configs, configcoef,T,eci):
    """
    Input: 
    corrs - Correlations
    vmat  - V-Matrix
    clusters - Maximal Cluster Information (multiplicity, longest neighbor length, no. of points)
    configs - Not used
    configcoef - Coefficients of subclusters - array containing the coeff of each subcluster
    T - Temperature
    eci - ECI's
    mu - chemical potentials
    
    Output:
    Vector representation gradient of F with Corrs
    [dF/dcorr0, dF/dcorr1, ...]
    """
    
    def get_kth_elem_jac(corrs, vmat, kb, clusters, configs, configcoef,corr_idx):
        
        dS_k = np.sum([kb[cluster_idx]*per_cluster_sum_jac(corrs,
                                                           vmat[cluster_idx],
                                                           configcoef[cluster_idx],
                                                           corr_idx,) 
                       for cluster_idx in configs.keys()
                      ])
        return dS_k
    
    def get_corrsum_jac(vmat,corrs):
        assert len(vmat) == len(corrs)
        corrsum = np.inner(vmat,corrs)
        if corrsum == 0:
            return np.NINF
            #corrsum = np.finfo(float).tiny

        return 1 + math.log(np.abs(corrsum))

    def per_cluster_sum_jac(corrs,vmat,configcoef,corr_idx):
        
        config_sum = np.sum([coef * vmat[config_idx][corr_idx] * get_corrsum_jac(vmat[config_idx],corrs) for config_idx, coef in enumerate(configcoef)
                            ])

        
        return config_sum
    
    #def get_dmu(vmat, mu, corr_idx):   
    #    return np.sum([mu[i]*vmat[1][i][corr_idx] for i, _ in enumerate(configcoef[1])])

    #dmu = np.array([get_dmu(vmat, mu, corr_idx) for corr_idx, _ in enumerate(corrs)])
    
    dH = np.array([cluster['mult']*eci[cluster_idx] for cluster_idx, cluster in clusters.items()])
    
    dS = np.array([get_kth_elem_jac(corrs, vmat, kb, clusters, configs, configcoef,corr_idx) for corr_idx, _ in enumerate(corrs)])
    
    return dH + kB*T*dS #- dmu

def F_jacobian_optim(corrs, vmat, kb, clusters, configs, configcoef,T,eci):

    dH = mults_eci 
    dS = all_vmat.T @ (multconfig_kb * (1 + np.log(np.abs(all_vmat @ corrs))))
    
    return dH + kB*T*dS

In [30]:
#f_jaco0 = F_jacobian(corrs0, vmat, kb, clusters, configs, configcoef,T,eci)
#f_jaco1 = F_jacobian(corrs1, vmat, kb, clusters, configs, configcoef,T,eci)
#f_jacosqs = F_jacobian(corrssqs, vmat, kb, clusters, configs, configcoef,T,eci)
#f_jacorand = F_jacobian(corrsrand, vmat, kb, clusters, configs, configcoef,T,eci)
#f_jacornd = F_jacobian(corrs_rnd, vmat, kb, clusters, configs, configcoef,T,eci)
frnd_jac = F_jacobian(corrs_rnd, vmat, kb, clusters, configs, configcoef,T,eci)
#frand = F(corrsrand, vmat, kb, clusters, configs, configcoef,T,eci)
fnew_jac = F_jacobian_optim(corrs_rnd, vmat, kb, clusters, configs, configcoef,T,eci)
#print(f"Corrs 0: {f_jaco0} \nCorrs 1: {f_jaco1} \nCorrs Rand {f_jacorand} \nCorrs SQS {f_jacosqs} \nCorrs RND {f_jacornd}")

In [31]:
frnd_jac, fnew_jac

(array([-11.088305,   0.000582,  -0.010316,   0.125114,  -0.04093 ,
          0.036542,   0.046397,   0.065335,  -0.040581,  -0.001179,
          0.000538,  -0.000283,   0.000173,   0.004662,   0.000425,
         -0.000313,  -0.001683,  -0.001697,  -0.001062,  -0.002441,
         -0.000056]),
 array([-11.088305,   0.000582,  -0.010316,   0.125114,  -0.04093 ,
          0.036542,   0.046397,   0.065335,  -0.040581,  -0.001179,
          0.000538,  -0.000283,   0.000173,   0.004662,   0.000425,
         -0.000313,  -0.001683,  -0.001697,  -0.001062,  -0.002441,
         -0.000056]))

## Set up Hessian

In [32]:
def F_hessian(corrs, vmat, kb, clusters, configs, configcoef,T,eci):
    """
    Input: 
    corrs - Correlations
    vmat  - V-Matrix
    clusters - Maximal Cluster Information (multiplicity, longest neighbor length, no. of points)
    configs - Not used
    configcoef - Coefficients of subclusters - array containing the coeff of each subcluster
    T - Temperature
    eci - ECI's
    
    Output:
    Vector representation gradient of F with Corrs
    [[d^2F/dcorr0 dcorr0, d^2F/dcorr0 dcorr1, ..., d^2F/dcorr0 dcorrn],
     [d^2F/dcorr1 dcorr0, d^2F/dcorr1 dcorr1, ..., d^2F/dcorr1 dcorrn],
     .
     .
     .
     [d^2F/dcorrn dcorr0, d^2F/dcorrn dcorr1, ..., d^2F/dcorrn dcorrn],
    ]
    """
    
    def get_corrsum_hess(vmat,corrs):
        assert len(vmat) == len(corrs)
        
        corrsum = np.inner(vmat,corrs)
        if corrsum == 0:
            return 1/np.PINF
            #corrsum = np.finfo(float).tiny

        return corrsum
    
    def get_config_val(corrs,vmat,configcoef,corr_idx_1,corr_idx_2):
        
        config_val = np.sum([coef * vmat[config_idx][corr_idx_1] * vmat[config_idx][corr_idx_2] / get_corrsum_hess(vmat[config_idx],corrs) 
                             for config_idx, coef in enumerate(configcoef)
                            ])
        
        return config_val
    
    def get_hessian_elem(corrs, vmat, kb, clusters, configs, configcoef,T,eci,corr_idx_1,corr_idx_2):
        
        hess_elem = np.sum([kb[cluster_idx] * get_config_val(corrs,
                                                             vmat[cluster_idx],
                                                             configcoef[cluster_idx],
                                                             corr_idx_1,
                                                             corr_idx_2
                                                            ) 
                            for cluster_idx in configs.keys()
                           ])
        return hess_elem
    
    d2F = np.empty([len(corrs),len(corrs)])
    
    d2F = np.array([[get_hessian_elem(corrs, vmat, kb, clusters, configs, configcoef, T, eci, corr_idx_1, corr_idx_2) for corr_idx_2, _ in enumerate(corrs)] for corr_idx_1, _ in enumerate(corrs)])
    
    return kB*T*d2F

def F_hessian_optim(corrs, vmat, kb, clusters, configs, configcoef,T,eci):
    
    t_0 = all_vmat @ corrs
    #d2S = ((multconfig_kb / t_0)[:, np.newaxis] * all_vmat).T @ all_vmat
    d2S = (np.diag(multconfig_kb / (all_vmat @ corrs)).T @ all_vmat).T @ all_vmat
    
    return kB*T*d2S
    

In [33]:
frnd_hess = F_hessian(corrs_rnd, vmat, kb, clusters, configs, configcoef,T,eci)
#frand = F(corrsrand, vmat, kb, clusters, configs, configcoef,T,eci)
fnew_hess = F_hessian_optim(corrs_rnd, vmat, kb, clusters, configs, configcoef,T,eci)

In [34]:
%%timeit
F_hessian(corrs_rnd, vmat, kb, clusters, configs, configcoef,T,eci)

72 ms ± 470 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [35]:
%%timeit
F_hessian_optim(corrs_rnd, vmat, kb, clusters, configs, configcoef,T,eci)

23.3 µs ± 244 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [36]:
np.allclose(frnd_hess, fnew_hess)

True

In [ ]:
def hessian(f):
    return jacfwd(jacrev(f))

H = hessian(F_optim_np)(corrs_rnd, vmat, kb, clusters, clustermult, configs, configcoef,T,eci)
print("hessian, with shape", H.shape)

In [ ]:
H.to_py()[-2]

In [ ]:
f_hess0 = F_hessian(corrs0, vmat, kb, clusters, configs, configcoef,T,eci)
f_hess1 = F_hessian(corrs1, vmat, kb, clusters, configs, configcoef,T,eci)
f_hessrand = F_hessian(corrsrand, vmat, kb, clusters, configs, configcoef,T,eci)
f_hesssqs = F_hessian(corrssqs, vmat, kb, clusters, configs, configcoef,T,eci)
f_hessrnd = F_hessian(corrs_rnd, vmat, kb, clusters, configs, configcoef,T,eci)
#print(f"Corrs 0:\n {f_hess0} \n Corrs 1:\n {f_hess1}\n Corrs Rand:\n {f_hessrand} \n Corrs SQS:\n {f_hesssqs} \n RND SQS:\n {f_hessrnd}")

# Constraints

In [20]:
#NOT BEING USED ANY MORE SINCE IT IS TRIVIALLY TRUE
def constraint_rhos_sum(corrs, vmat, clusters, configcoef,):
    """
    Constraints the sum of each rho. As of now, it's done in a weird way, where the total sum of the array:
    [1 - sum(rho), .... ] is constrained to sum to 0. This along with the constraint that each rho is between
    0 and 1, seems to make it work. I think that by the this might be a redundant constraint as well.
    """
    rho_sum = []

    def clus_prob(cluster_idx):
        rho = np.matmul(vmat[cluster_idx],corrs)
        return rho
    
    for cluster_idx, _ in clusters.items():
        rho = clus_prob(cluster_idx)
        rho_sum.append(np.sum(configcoef[cluster_idx]*rho))
    
    return np.sum(1 - np.array(rho_sum))

def constraint_singlet(corrs,FIXED_CORR_1):
    """
    constrains the 1-pt correlation:
    corrs[1] = FIXED_CORR_1
    """
    return corrs[1] - FIXED_CORR_1   

def constraint_zero(corrs):
    """
    constrains the 1-pt correlation:
    corrs[0] = 1
    """
    return 1 - corrs[0]

def constraint_hessian(corrs, vmat, kb, clusters, configs, configcoef,T,eci):
    hessian_F = hessian(F)
    return np.amin(np.real(eigvals(hessian_F(corrs,vmat, kb, clusters, configs, configcoef,T,eci))))

def constraint_NN(corrs,FIXED_CORR_2):
    """
    constrains the 2-pt correlation:
    corrs[2] = FIXED_CORR_2
    """
    return corrs[2] - FIXED_CORR_2 

class MyBounds:
    """
    Class to constrain the trial correlations of Basin Hopping
    """
    def __init__(self,xmax=[1]*6, xmin=[-1]*6,):
        self.xmax = np.array(xmax)
        self.xmin = np.array(xmin)
        
    def __call__(self, **kwargs):
        x = kwargs["x_new"]
        tmax = bool(np.all(x <= self.xmax))
        tmin = bool(np.all(x >= self.xmin))

        return tmax and tmin
    
class MyTakeStep:
    
    def __init__(self, vmat, clusters, stepsize=0.1):
        self.stepsize = stepsize
        self.vmat = vmat
        self.clusters = clusters
        self.rng = np.random.default_rng()
    
    def __call__(self, x):
        s = self.stepsize
        
        validcorr = np.ones(len(self.clusters), dtype=bool)
        
        for _ in iter(int,1):
            x_trial = x + self.rng.uniform(-s, s, x.shape)
            for cluster_idx, _ in self.clusters.items():
                rho = np.matmul(self.vmat[cluster_idx],x_trial)
                validcorr[cluster_idx] = np.all(rho >= 0)
            if bool(np.all(validcorr)):
                break
            
        return x_trial

# Initial Sampling

In [336]:
FIXED_CORR1 = 0.0
            
for c in iter(int,1): #infinite loop till a valid starting correlations are found
    corr0 = 1.0
    corr1 = FIXED_CORR1
    validcorr = np.ones(len(clusters), dtype=bool)
    if corr1 <=0 :
        corr2 = np.random.uniform(-2*corr1 - 1, 1)
        corr3 = np.random.uniform(-2*corr1 - 1, 1)
        #corr4 = np.random.uniform(-corr1+corr3-1, 1)
    else:
        corr2 = np.random.uniform(2*corr1 - 1, 1)
        corr3 = np.random.uniform(2*corr1 - 1, 1)
        #corr4 = np.random.uniform(corr1+corr3-1, 1)

    corr4 = np.random.uniform(corr1+corr2-1,corr1-corr2+1)
    corr5 = np.random.uniform(2*corr3-1,1)
    corrs0 = np.array([1, corr1, corr2, corr3, corr4, corr5])
    for cluster_idx, _ in clusters.items():
        rho = np.matmul(vmat[cluster_idx],corrs0)
        validcorr[cluster_idx] = np.all(rho >= 0)
    
    if bool(np.all(validcorr)):
        print(corrs0)
        print(c)
        break 


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 6 is different from 21)

In [22]:
def get_valid_corrs(FIXED_CORR1,FIXED_CORR2=None):
    
    for _ in iter(int,1):
        corr0 = 1.0
        corr1 = FIXED_CORR1
        validcorr = np.ones(len(clusters), dtype=bool)
        if corr1 <=0 :
            if FIXED_CORR2 is None:
                corr2 = np.random.uniform(-2*corr1 - 1, 1)
            else:
                corr2 = FIXED_CORR2
            corr3 = np.random.uniform(-2*corr1 - 1, 1)
        else:
            if FIXED_CORR2 is None:
                corr2 = corr2 = np.random.uniform(2*corr1 - 1, 1)
            else:
                corr2 = FIXED_CORR2
            corr3 = np.random.uniform(2*corr1 - 1, 1)

        corr4 = np.random.uniform(corr1+corr2-1,corr1-corr2+1)
        corr5 = np.random.uniform(2*corr3-1,-2*corr1 + 2*corr4 + 1)
        corrs0 = np.array([1, corr1, corr2, corr3, corr4, corr5])

        for cluster_idx, _ in clusters.items():
            rho = np.matmul(vmat[cluster_idx],corrs0)
            validcorr[cluster_idx] = np.all(rho >= 0)
        
        if bool(np.all(validcorr)):
            print(corrs0)
            break 
    return corrs0

In [464]:
#corrs0 = get_valid_corrs(np.random.uniform(-1,1))


[ 1.        0.        0.        0.002576 -0.000744 -0.019188 -0.000265
  0.000602  0.024632 -0.001924  0.003015 -0.000347 -0.011687  0.011428
  0.007519  0.00791  -0.009094  0.014028 -0.014019  0.005869  0.021905]


In [465]:
for vm in vmat.values():
    print(f"{' '.join(map(str, vm @ corrs0))}")
corrs_rnd=corrs0

1.0
0.33333 0.33333 0.33333
0.11139616423913207 0.11089526569817265 0.11103854430765622 0.10965427474957588 0.11278045955225148 0.10951099614009231
0.11108054043298837 0.11118269064022583 0.11106677157817334 0.11309729456670795 0.10905001479306622 0.11321321362876044
0.037292197336296885 0.036955686665568255 0.03715828023726691 0.03768354984532849 0.03626602918727589 0.037624234883113394 0.037226312204636784 0.03606859709523387 0.03761035639830199 0.036900090820371614 0.03669539495684884 0.03848490007422216 0.03657200248562808 0.03816841290244469 0.036308128919583425 0.03852389420219808 0.036098344324730226 0.03711433101865711
0.012663312396959224 0.012273265638644427 0.012365726810314559 0.013032586103279257 0.011930470340627138 0.012286048948207947 0.011799631954419063 0.012892947394527385 0.012039448629734656 0.012239637233776959 0.01304582468444182 0.011909701201837954 0.012621617726319147 0.012106048358589595 0.012302512127894677 0.012416626355868168 0.012454099655047119 0.0134583

# Uniform Sampling

In [25]:
np.linspace(-1, 1, num=7)

array([-1.  , -0.67, -0.33,  0.  ,  0.33,  0.67,  1.  ])

In [29]:
eci

{0: 0, 1: 0.0, 2: 0.04, 3: 0, 4: 0, 5: 0}

In [35]:
corrs0

array([ 1.  , -0.5 ,  0.59,  0.01,  0.15, -0.01])

In [39]:
#results_uniform = pd.DataFrame(columns = ['T', '1-point_corr', 'F','corrs'])
NUM_TRIALS = 50
MAX_TEMP = 1

for temp in [100]:#tqdm(np.linspace(0, MAX_TEMP, num=11)):
    for x in [-0.5]:#tqdm(np.linspace(-1+np.finfo(float).eps,1-np.finfo(float).eps,9)):
        FIXED_CORR_1 = x
        
        MIN_RES_VAL = 1e5 #random large number
        rho_pair = np.array([None,None])
        
        MIN_RES = corrs0

        linear_constraints = []

        for cluster_idx, _ in clusters.items():
            
            if cluster_idx == 0:
                linear_constraints.append(LinearConstraint(vmat[cluster_idx],
                                                           [1]*len(configcoef[cluster_idx]),
                                                           [1]*len(configcoef[cluster_idx])))
            else:
                linear_constraints.append(LinearConstraint(vmat[cluster_idx],
                                                           [0]*len(configcoef[cluster_idx]),
                                                           [1]*len(configcoef[cluster_idx])))
        
        bounds_corrs = Bounds([1, FIXED_CORR_1,*[-1]*(len(clusters)-2)],
                              [1, FIXED_CORR_1,*[1]*(len(clusters)-2)]
                             )
     
        options = {'verbose' : 1,
                   'maxiter' : 1000,
                   'xtol'    : 1e-18,
                   'gtol'    : 1e-18,
                   'initial_constr_penalty' : 10,
                  }
        #corrs0 = np.array([1, x, *np.random.uniform(-1, 1, 4)])
        corrs0 = np.array([ 1.,     -0.5,     0.25,    0.25,   -0.125,   0.0625])
        for _ in tqdm(range(NUM_TRIALS)):
            jitter = np.array([0, 0, *np.random.normal(0, .001, corrs0[2:].shape)])
            corrstrial = corrs0+jitter  
            res = minimize(F,
                           corrstrial,
                           method='trust-constr',
                           args=(vmat, kb, clusters, configs, configcoef,temp,eci),
                           options=options,
                           jac='3-point',
                           hess=BFGS(),
                           constraints=[*linear_constraints, 
                                        {'fun': constraint_singlet, 'type': 'eq', 'args': [FIXED_CORR_1],'hess':0},
                                        {'fun': constraint_zero, 'type':'eq',},
                                       ],
                           bounds=bounds_corrs,
                          )
            
            if res.fun < MIN_RES_VAL:
                MIN_RES = res
                MIN_RES_VAL = res.fun
                print(f"Found new minimum for x:{x}, T:{temp} fun: {MIN_RES_VAL}")
                print(f'Current minimum correlations: {res.x}')
                for cluster_idx in clusters.keys():
                    print(np.matmul(vmat[cluster_idx],res.x))
        
            
        for cluster_idx in clusters.keys():
                assert np.isclose(np.inner(configcoef[cluster_idx],np.matmul(vmat[cluster_idx],res.x)),1.0)
        
        results_uniform = results_uniform.append({'T' : temp, 
                                           '1-point_corr' : x, 
                                           'F' : MIN_RES.fun, 
                                           'corrs': MIN_RES.x,
                                          }, 
                                          ignore_index = True
                                         )
        
now = datetime.now()
now = now.strftime("%d-%m-%H:%M")
#results_uniform.to_pickle(f'results/uni_{eci[2]},{eci[3]}_{MAX_TEMP}.pickle')

  0%|          | 0/50 [00:00<?, ?it/s]

`xtol` termination condition is satisfied.
Number of iterations: 521, function evaluations: 8138, CG iterations: 509, optimality: 1.70e-05, constraint violation: 0.00e+00, execution time:  5.7 s.
Found new minimum for x:-0.5, T:100 fun: -0.017157678971778927
Current minimum correlations: [ 1.   -0.5   0.    0.38  0.12 -0.24]
[1.]
[0.75 0.25]
[0.5  0.25 0.  ]
[0.6  0.15 0.1 ]
[0.35 0.15 0.1  0.25 0.   0.  ]
[0.19 0.15 0.1  0.   0.   0.  ]
`xtol` termination condition is satisfied.
Number of iterations: 351, function evaluations: 5421, CG iterations: 339, optimality: 1.93e-05, constraint violation: 0.00e+00, execution time:  3.6 s.
`xtol` termination condition is satisfied.
Number of iterations: 407, function evaluations: 6981, CG iterations: 395, optimality: 2.32e-05, constraint violation: 0.00e+00, execution time:  4.3 s.
`xtol` termination condition is satisfied.
Number of iterations: 374, function evaluations: 5785, CG iterations: 362, optimality: 1.18e-06, constraint violation: 0.00

KeyboardInterrupt: 

In [38]:
eci

{0: 0, 1: 0.0, 2: 0.04, 3: 0, 4: 0, 5: 0}

## Plot Phase Diagram

In [ ]:
fig = go.Figure()

for T in results_uniform['T'].unique():
    fig.add_trace(go.Scatter(x = results_uniform[results_uniform['T'] == T]['1-point_corr'],
                             y = results_uniform[results_uniform['T'] == T]['F'],
                             mode='markers+lines',
                             name=f'T = {T}',
                            )
                 )

fig.update_layout(
    title="F vs 1-point Corr",
    xaxis_title="1-point Corr",
    yaxis_title="F",
    legend_title="Temperature",
    template='seaborn'
)
#fig.update_traces(texttemplate='%{text:.2s}', textposition='top center')
fig.show()

## Plot Pair Correlation

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x = results_uniform[results_uniform['1-point_corr'] == 0.0]['T'],
                         y = results_uniform[results_uniform['1-point_corr'] == 0.0]['corrs'].str[2],
                         mode='markers+lines',
                        )
             )

fig.update_layout(
    title="T vs 2-point Corr",
    xaxis_title="T",
    yaxis_title="2-point Corr",
    template='seaborn'
)
#fig.update_traces(texttemplate='%{text:.2s}', textposition='top center')
fig.show()

In [ ]:
np.linspace(-1+np.finfo(float).eps,1-np.finfo(float).eps,19)

# Basin Hopping

## Defining the Callback

In [ ]:
def basin_hopping_callback(corrs, G, accept):
    """
    Function to print diagnostic while fitting. 
    Not being used right now
    """
    if accept == True:
        clear_output(wait=True)
        print(f'1-pt Correlation: {corrs[1]:.2f}')
        print(f'Concentation: {(corrs[1] - (-1))/(1 - (-1)):.2f}')
        print(f'New Minima found --> G: {G:.2f}')
        print('Current Rho:')
        for cluster_idx in clusters:
            print(np.matmul(vmat[cluster_idx],corrs))
        print("===========================")
        
        

## Optimisation Code

In [ ]:
results_basinhopping = pd.DataFrame(columns = ['T', '1-point_corr', 'F','corrs'])
MAX_TEMP = 500
now = datetime.now()
now = now.strftime("%d-%m-%H-%M")

for temp in tqdm(np.linspace(0, MAX_TEMP, num=21)):
    for x in [0]:#tqdm(np.linspace(-1+np.finfo(float).eps,1-np.finfo(float).eps,9)):

        FIXED_CORR_1 = x

        rho_pair = np.array([None,None])
        
        #Make a first guess
        for _ in iter(int,1): #infinite loop till a valid starting correlations are found
            corrs0 = np.array([1, x, *np.random.uniform(-1, 1, 4)])
            validcorr = np.ones(len(clusters), dtype=bool)

            for cluster_idx, _ in clusters.items():
                rho = np.matmul(vmat[cluster_idx],corrs0)
                validcorr[cluster_idx] = np.all(rho >= 0)

            if bool(np.all(validcorr)):
                break   
        
        #corrs0 = np.array([1, x, *np.random.uniform(-1, 1, 4)])
        linear_constraints = []

        #Linear Constraint for each rho to be between 0 and 1
        for cluster_idx, _ in clusters.items():
            if cluster_idx == 0:
                linear_constraints.append(LinearConstraint(vmat[cluster_idx],
                                                           [1]*len(configcoef[cluster_idx]),
                                                           [1]*len(configcoef[cluster_idx])))
            else:
                linear_constraints.append(LinearConstraint(vmat[cluster_idx],
                                                           [0]*len(configcoef[cluster_idx]),
                                                           [1]*len(configcoef[cluster_idx])))
        
        #Set bounds for the local minimisation 
        #limit correlations to [1, 1-pt, [-1,1], [-1,1], ...]
        bounds_corrs = Bounds([1, FIXED_CORR_1,*[-1]*(len(clusters)-2)],
                              [1, FIXED_CORR_1,*[1]*(len(clusters)-2)],
                             )
     
        options = {'verbose' : 0,
                   'maxiter' : 5000,
                   'xtol'    : 1e-9,
                   'initial_constr_penalty' : 10,
                  }
        
        minimizer_kwargs = {'args':(vmat, kb, clusters, configs, configcoef,temp,eci),
                            'method': 'trust-constr',
                            'options': options,
                            'jac': F_jacobian, 'hess': F_hessian,
                            'constraints' : [*linear_constraints, 
                                             {'fun': constraint_singlet, 'type': 'eq', 'args': [FIXED_CORR_1], 'hess': 0},
                                             {'fun': constraint_zero, 'type':'eq','hess': 0},
                                             #{'fun': constraint_rhos_sum, 'type': 'eq', 'args': [vmat, clusters, configcoef,]},
                                            ],
                            'bounds': bounds_corrs,
                           }
        
        #Bounds for trial correlations
        mybounds = MyBounds(xmax=[1, FIXED_CORR_1,*[1]*(len(clusters)-2)], 
                            xmin=[1, FIXED_CORR_1,*[-1]*(len(clusters)-2)]
                           )
        
        mytakestep = MyTakeStep(vmat,
                                clusters,
                                stepsize=0.05
                               )
        
        res = basinhopping(F, 
                           corrs0, #first guess
                           niter=1000, #total num of iterations
                           T=0.01, #temp for Metropolis MC trial search
                           #stepsize=0.1,
                           minimizer_kwargs=minimizer_kwargs,
                           niter_success=10, #num iters to exit after no new minima found 
                           interval=5, #num iters to change step size
                           disp=True,
                           #accept_test=mybounds,
                           take_step=mytakestep,
                           #seed=42,
                           #callback=basin_hopping_callback
                          )

        #Code to extract rhos and check if they sum them to 1 for sanity. Not used.
        for cluster_idx in clusters.keys():
                assert np.isclose(np.inner(configcoef[cluster_idx],np.matmul(vmat[cluster_idx],res.x)),1.0)
        
        results_basinhopping = results_basinhopping.append({'T' : temp, 
                                                     '1-point_corr' : x, 
                                                     'F' : res.fun, 
                                                     'corrs': res.x,
                                                    }, 
                                                    ignore_index = True
                                                   )

#save results
results_basinhopping.to_pickle(f'results/bh_{eci[2]}_{MAX_TEMP}.pickle')

## Plot Phase Diagram

In [ ]:
fig = go.Figure()

for T in results_basinhopping['T'].unique():
    fig.add_trace(go.Scatter(x = results_basinhopping[results_basinhopping['T'] == T]['1-point_corr'],
                             y = results_basinhopping[results_basinhopping['T'] == T]['F'],
                             mode='lines+markers',
                             name=f'T = {T}',
                            )
                 )

fig.update_layout(
    title="F vs 1-point Corr",
    xaxis_title="1-point Corr",
    yaxis_title="F",
    legend_title="Temperature",
    template='seaborn'
)
#fig.update_traces(texttemplate='%{text:.2s}', textposition='top center')
fig.show()

## Plot Pair Correlation



In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x = results_basinhopping[results_basinhopping['1-point_corr'] == 0.0]['T'],
                         y = results_basinhopping[results_basinhopping['1-point_corr'] == 0.0]['corrs'].str[2],
                         mode='markers+lines',
                        )
             )

fig.update_layout(
    title="T vs 2-point Corr",
    xaxis_title="T",
    yaxis_title="2-point Corr",
    template='seaborn'
)
#fig.update_traces(texttemplate='%{text:.2s}', textposition='top center')
fig.show()

# Fix 1-point and NN

In [ ]:
def singlet_doublet_validpts(triangle = np.array([
    [0, 0],
    [-1, 1],
    [1, 1],
])):
    
    def uniform_triangle(u, v):
        while True:
            s = random.random()
            t = random.random()
            in_triangle = s + t <= 1
            p = s * u + t * v if in_triangle else (1 - s) * u + (1 - t) * v
            yield p

    it = uniform_triangle(
        triangle[1] - triangle[0],
        triangle[2] - triangle[0],
    )

    points = np.array(list(itertools.islice(it, 0, 100)))
    points += triangle[0]
    
    return points

In [ ]:
def get_uniform_traingular_points(xvalues, yvalues, x1=0, y1=0, x2=-1, y2=1, x3=1, y3=1, num = 10):
    
    def isInside(x, y, x1=0, y1=0, x2=-1, y2=1, x3=1, y3=1,):
    
        def area(x1, y1, x2, y2, x3, y3):
            return abs((x1*(y2 - y3) + x2*(y3 - y1) + x3*(y1 - y2))/2.0)

        # Calculate area of triangle ABC
        A = area(x1, y1, x2, y2, x3, y3)
        # Calculate area of triangle PBC
        A1 = area(x, y, x2, y2, x3, y3)
        # Calculate area of triangle PAC
        A2 = area(x1, y1, x, y, x3, y3)
        # Calculate area of triangle PAB
        A3 = area(x1, y1, x2, y2, x, y)
        # Check if sum of A1, A2 and A3
        # is same as A
        if(A == A1 + A2 + A3):
            return True
        else:
            return False
                                   

    points = []
    #xx, yy = np.meshgrid(xvalues, yvalues)
    grid = np.meshgrid(xvalues, yvalues)
    grid = np.vstack(list(map(np.ravel, grid))).T

    for x, y in grid:
        if isInside(x,y,x1=x1,y1=y1,x2=x2,y2=y2,x3=x3,y3=y3):
            points.append([x,y])
    
    return np.array(points)

In [ ]:
#singlet_doublet_pairs = singlet_doublet_validpts()
num = 8
xvalues = np.linspace(-1+np.finfo(float).eps,1-np.finfo(float).eps,num)
yvalues = np.linspace(0,1,num)
singlet_doublet_pairs_1 = get_uniform_traingular_points(xvalues, yvalues, x1=0, y1=0, x2=-1, y2=1, x3=1, y3=1, num = num)

xvalues = np.linspace(-1+np.finfo(float).eps,1-np.finfo(float).eps,num)
yvalues = np.linspace(0,-1,num)
singlet_doublet_pairs_2 = get_uniform_traingular_points(xvalues, yvalues, x1=0, y1=0, x2=-1, y2=-1, x3=1, y3=-1,num=num)

xvalues = np.linspace(-1+np.finfo(float).eps,1-np.finfo(float).eps,num)
yvalues = np.linspace(-1+np.finfo(float).eps,1-np.finfo(float).eps,num)
singlet_doublet_pairs_3 = get_uniform_traingular_points(xvalues, yvalues, x1=0, y1=-1, x2=-1, y2=1, x3=1, y3=1,num=num)
singlet_doublet_pairs_3 = np.append(singlet_doublet_pairs_3,[[0,-1+np.finfo(float).eps]],axis=0)

singlet_doublet_pairs = np.vstack([np.array(singlet_doublet_pairs_1),np.array(singlet_doublet_pairs_2)])
plt.style.use('default')
plt.scatter(singlet_doublet_pairs_3[:, 0], singlet_doublet_pairs_3[:, 1], s=5)
print(len(singlet_doublet_pairs_3))
plt.show()

In [ ]:
results_blanket = pd.DataFrame(columns = ['T', '1-point_corr', '2-point_corr', 'F','corrs'])
NUM_TRIALS = 50
MAX_TEMP = 500

#corrsrange = np.linspace(-1+np.finfo(float).eps,1-np.finfo(float).eps,21)
#singlet_doublet_pairs = singlet_doublet_validpts()
#singlet_doublet_pairs = get_uniform_traingular_points(num=20)
print(f'Working ECIs: {eci}')
for temp in tqdm(np.linspace(0, MAX_TEMP, num=11)):
    for FIXED_CORR_1, FIXED_CORR_2 in tqdm(singlet_doublet_pairs_3):
        
        print('\n=============================================================')
        print(f'Working on 1-point corr: {FIXED_CORR_1:.4f}, NN corr: {FIXED_CORR_2:.4f}, Temp: {temp}')
        print('=============================================================\n')

        MIN_RES_VAL = 1e5 #random large number
        MIN_RES = corrs0

        linear_constraints = []

        for cluster_idx, _ in clusters.items():

            if cluster_idx == 0:
                linear_constraints.append(LinearConstraint(vmat[cluster_idx],
                                                           [1]*len(configcoef[cluster_idx]),
                                                           [1]*len(configcoef[cluster_idx])))
            else:
                linear_constraints.append(LinearConstraint(vmat[cluster_idx],
                                                           [0]*len(configcoef[cluster_idx]),
                                                           [1]*len(configcoef[cluster_idx])))

        bounds_corrs = Bounds([1, FIXED_CORR_1, FIXED_CORR_2, *[-1]*(len(clusters)-3)],
                              [1, FIXED_CORR_1, FIXED_CORR_2, *[1]*(len(clusters)-3)]
                             )

        options = {'verbose' : 0,
                   'maxiter' : 3000,
                   'xtol'    : 1e-15,
                   'initial_constr_penalty' : 10,
                  }

        for _ in tqdm(range(NUM_TRIALS)):

            corrs0 = np.array([1, FIXED_CORR_1, FIXED_CORR_2, *np.random.uniform(-1, 1, len(clusters)-3)])

            res = minimize(F,
                           corrs0,
                           method='trust-constr',
                           args=(vmat, kb, clusters, configs, configcoef,temp,eci),
                           options=options,
                           #jac='3-point',
                           #hess=BFGS(),
                           jac=F_jacobian,
                           hess=F_hessian,
                           constraints=[*linear_constraints, 
                                        {'fun': constraint_singlet, 'type': 'eq', 'args': [FIXED_CORR_1]},
                                        {'fun': constraint_zero, 'type':'eq',},
                                        {'fun': constraint_NN, 'type':'eq','args':[FIXED_CORR_2]}
                                       ],
                           bounds=bounds_corrs,
                          )

            if res.fun < MIN_RES_VAL:
                MIN_RES = res
                MIN_RES_VAL = res.fun
                print(f"Found new minimum for Corr1:{FIXED_CORR_1:.4f}, Corr2:{FIXED_CORR_2:.4f} fun: {MIN_RES_VAL:.15f}")
                print(f'Current minimum correlations: {res.x}')
#                     for cluster_idx, _ in clusters.items():
#                         rho = np.matmul(vmat[cluster_idx],res.x)
#                         print(rho)


        break_next = False
        for cluster_idx in clusters.keys():
            try:
                assert np.isclose(np.inner(configcoef[cluster_idx],np.matmul(vmat[cluster_idx],res.x)),1.0)
            except AssertionError:
                break_next = True
        if break_next:
            print('No valid solution found')
            continue

        results_blanket = results_blanket.append({'T' : temp, 
                                           '1-point_corr' : FIXED_CORR_1,
                                           '2-point_corr' : FIXED_CORR_2,
                                           'F' : MIN_RES.fun, 
                                           'corrs': MIN_RES.x,
                                          }, 
                                          ignore_index = True
                                         )
        #print(results_blanket)
now = datetime.now()
now = now.strftime("%d-%m-%H:%M")
results_blanket.to_pickle(f'results/blanket_{eci[2]}_{eci[3]}_{MAX_TEMP}.pickle')
results_blanket.to_pickle('temp.pickle')

In [ ]:
now = datetime.now()
now = now.strftime("%d")
results_blanket.to_pickle(f'results/blanket_{eci[2]}_{MAX_TEMP}_{now}.pickle')

In [ ]:
results_blanket['T'].unique()

In [ ]:
temp = 1000
results_blanket[
                (results_blanket['T'] == temp)
               ]

Xs = results_blanket[results_blanket['T'] == temp]['1-point_corr'].values
Ys = results_blanket[results_blanket['T'] == temp]['2-point_corr'].values
Zs = results_blanket[results_blanket['T'] == temp]['F'].values

fig, ax2 = plt.subplots(nrows=1,figsize=(8, 6), dpi=100)
plt.style.use('ggplot')
ax2.set_title(f'T = {temp}')
ax2.tricontour(Xs, Ys, Zs, levels=10, linewidths=0.5,)
cntr2 = ax2.tricontourf(Xs, Ys, Zs, levels=14,)

fig.colorbar(cntr2, ax=ax2,label='F')
ax2.plot(Xs, Ys, 'ko', ms=3)
ax2.set(xlim=(-1, 1), ylim=(-1, 1))

plt.xlabel("1-point Correlation")
plt.ylabel("2-point NN Correlation")
plt.subplots_adjust(hspace=0.5)

plt.show()

In [ ]:
Xs

In [ ]:
results_blanket['T'].unique()

In [ ]:
fig = go.Figure()
#fig.add_trace(traces)
fig.update_layout(
    xaxis_title="1-point Corr",
    yaxis_title="2-point Corr",
    template='ggplot2',
    legend_title="Legend Title",
)
for temp in [0,150,200,300,450]:#[0.,  50., 100., 150., 200., 250., 300., 350., 400., 450., 500.]:
    Xs = results_blanket[results_blanket['T'] == temp]['1-point_corr'].values
    Ys = results_blanket[results_blanket['T'] == temp]['2-point_corr'].values
    Zs = results_blanket[results_blanket['T'] == temp]['F'].values
    points2D = np.vstack([Xs,Ys]).T
    tri = Delaunay(points2D)

    simplices = tri.simplices
    temp_trace = ff.create_trisurf(x=Xs, y=Ys, z=Zs,
                                   simplices=simplices,
                                   title="F vs singlet-NN-corrlations",
                                   show_colorbar=False,
                                   width=1000,
                                   height=1000,
                                   #aspectratio=dict(x=1, y=1, z=10)
                                  )
    fig.add_traces([temp_trace.data[0]])

fig.layout.scene.xaxis.title='1-point Correlation'
fig.layout.scene.yaxis.title='2-point Correlation'
fig.layout.scene.zaxis.title='F'

fig.update_layout(
    autosize=False,
    width=750,
    height=750,
)


fig.show()

# Fixed Chemical Potentials

In [ ]:
results_mu = pd.DataFrame(columns = ['T', 'mu', 'F','corrs'])
NUM_TRIALS = 50
MAX_TEMP = 1

for temp in [0]:#tqdm(np.linspace(0, MAX_TEMP, num=11)):
    for mu_1 in [0.25]:
        
        mu = np.array([-mu_1,mu_1])

        MIN_RES_VAL = 1e5 #random large number
        rho_pair = np.array([None,None])
        MIN_RES = corrs0

        linear_constraints = []

        for cluster_idx, _ in clusters.items():
            
            if cluster_idx == 0:
                linear_constraints.append(LinearConstraint(vmat[cluster_idx],
                                                           [1]*len(configcoef[cluster_idx]),
                                                           [1]*len(configcoef[cluster_idx])))
            else:
                linear_constraints.append(LinearConstraint(vmat[cluster_idx],
                                                           [0]*len(configcoef[cluster_idx]),
                                                           [1]*len(configcoef[cluster_idx])))
        
        bounds_corrs = Bounds([1, *[-1]*(len(clusters)-1)],
                              [1, *[1]*(len(clusters)-1)]
                             )
     
        options = {'verbose' : 1,
                   'maxiter' : 1000,
                   'xtol'    : 1e-12,
                   'initial_constr_penalty' : 10,
                  }
        
        for _ in tqdm(range(NUM_TRIALS)):
            
            corrs0 = np.array([1.   , *np.random.uniform(-1, 1, 5)])             
#             for _ in iter(int,1): #infinite loop till a valid starting correlations are found
#                 corrs0 = np.array([1, x, *np.random.uniform(-1, 1, len(clusters)-2)])
#                 validcorr = np.ones(len(clusters), dtype=bool)
        
#                 for cluster_idx, _ in clusters.items():
#                     rho = np.matmul(vmat[cluster_idx],corrs0)
#                     validcorr[cluster_idx] = np.all(rho >= 0)
                
#                 if bool(np.all(validcorr)):
#                     break   
                
            res = minimize(F,
                           corrs0,
                           method='trust-constr',
                           args=(vmat, kb, clusters, configs, configcoef,temp,eci,mu),
                           options=options,
                           #jac='3-point',
                           #hess=BFGS(),
                           jac=F_jacobian,
                           hess=F_hessian,
                           constraints=[*linear_constraints, 
                                        #{'fun': constraint_singlet, 'type': 'eq', 'args': [FIXED_CORR_1]},
                                        {'fun': constraint_zero, 'type':'eq',},
                                       ],
                           bounds=bounds_corrs,
                          )
            
            if res.fun < MIN_RES_VAL:
                MIN_RES = res
                MIN_RES_VAL = res.fun
                print(f"Found new minimum for mu:{mu}, T:{temp} fun: {MIN_RES_VAL}")
                print(f'Current minimum correlations: {res.x}')
        
            
        for cluster_idx in clusters.keys():
                assert np.isclose(np.inner(configcoef[cluster_idx],np.matmul(vmat[cluster_idx],res.x)),1.0)
        
        results_mu = results_mu.append({'T' : temp, 
                                           'mu' : mu_1, 
                                           'F' : MIN_RES.fun, 
                                           'corrs': MIN_RES.x,
                                          }, 
                                          ignore_index = True
                                         )
        
now = datetime.now()
now = now.strftime("%d-%m-%H:%M")
#results_uniform.to_pickle(f'results/uni_{eci[2]},{eci[3]}_{MAX_TEMP}.pickle')

In [ ]:
Found new minimum for mu:[-0.25  0.25], T:0 fun: -0.14929162153260508
Current minimum correlations: [1.   0.99 0.97 0.97 0.96 0.95]
Found new minimum for mu:[0 0], T:0 fun: -0.05998629111609842
Current minimum correlations: [ 1.  0. -0. -1. -0.  1.]  

In [ ]:
#corrs0 = np.array([1.   , *np.random.uniform(-1, 1, 5)])             
for _ in iter(int,1): #infinite loop till a valid starting correlations are found
    corrs0 = np.array([1, *np.random.uniform(-1, 1, 5)])
    validcorr = np.ones(len(clusters), dtype=bool)

    for cluster_idx, _ in clusters.items():
        rho = np.matmul(vmat[cluster_idx],corrs0)
        validcorr[cluster_idx] = np.all(rho >= 0)

    if bool(np.all(validcorr)):
        break
print(corrs0)